In [15]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [16]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\bethg\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


## Visit the NASA mars news site

In [15]:
# Visit the Mars news site
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [16]:
# Convert the browser html to a soup object
html = browser.html
soup = bs(html, 'html.parser')

In [17]:
# Use the parent element to find the first a tag and save it as `news_title`
element = soup.select_one('div.list_text')
element.find('div', class_='content_title')
news_title = element.find('div', class_='content_title').get_text()
news_title

"NASA's InSight Mars Lander Gets a Power Boost"

In [18]:
# Use the parent element to find the paragraph text
news_p = element.find('div', class_='article_teaser_body').get_text()
news_p

'The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.'

## JPL Space Images Featured Image

In [21]:
# Visit URL
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)

In [23]:
# Find and click the full image button
#.click
variable_button = browser.find_by_tag('button')[1]
variable_button.click()

In [24]:
# Parse the resulting html with soup
html_page = browser.html
html_soup = bs(html_page, 'html.parser')
print(html_soup)

<html class="fancybox-margin fancybox-lock"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-t

In [26]:
# find the relative image url
featured_image_url = html_soup.find('img', class_ = 'fancybox-image').get('src')
featured_image_url

'image/featured/mars2.jpg'

In [27]:
# Use the base url (prefix webpage) to create an absolute url
jpeg_html = ('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + featured_image_url)
print(jpeg_html)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


## Mars Facts

In [37]:
# Use `pd.read_html` to pull the data from the Mars-Earth Comparison section
mars_df = pd.read_html('https://space-facts.com/mars/')[0]
mars_df.columns = ['Description', 'Mars']
mars_df.set_index('Description', inplace=True)
#mars_df.style.set_caption('Mars Facts')
mars_df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
mars_df.to_html('beths_table.html')

## Hemispheres

In [19]:
#scrape mars hemispher title and images
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

hemisphere_image_urls = []

for i in range (4):
    #looks like an active person is using the webpage and not look like scraping
    time.sleep(15)
    images=browser.find_by_tag('h3')
    images[i].click()
    html=browser.html
    soup=bs(html, 'html.parser')
    #<img class="wide-image" src="/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg">
    ending_url = soup.find('img', class_ = 'wide-image')['src']
    #<h2 class="title">Cerberus Hemisphere Enhanced</h2>
    image_title = soup.find('h2', class_='title').text
    image_url = f'https://astrogeology.usgs.gov{ending_url}'
    image_dict = {'title': image_title, 'image_url': image_url}
    hemisphere_image_urls.append(image_dict)
    browser.back()

browser.quit()
    

In [23]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]